# Creation of Be stars database from catalogs of variable stars#
(see Zebrun et al. (2001), [Sabogal et al. (2005)](http://cdsads.u-strasbg.fr/cgi-bin/nph-data_query?bibcode=2005MNRAS.361.1055S&db_key=AST&link_type=ABSTRACT&high=5c40bd528c12691)) and OGLE archive)

I want to show you, howto create database of interestings astrophysical objects for scientfic study. Namely, we will query online database (Vizier,OGLE,ASAS etc.), select objects according predefined selection criteria, download source data and store result to local database file.

As the example, I will select *Be* stars from *Vizier* database according catalog, published by [Sabogal et al. (2005)](http://cdsads.u-strasbg.fr/cgi-bin/nph-data_query?bibcode=2005MNRAS.361.1055S&db_key=AST&link_type=ABSTRACT&high=5c40bd528c12691). They used Zebrun catalog of variable stars, and select *Be* stars candidates using photometric criteria. Zebrun catalog  is represented by *Table zebruncatalog* and Sabogal catalog of Be stars by *Table sabogalcatalog*. SQL database contains both catalogs, and is stored in the file *test.db*

In [1]:
import os
import ipywidgets as widgets
from sqlalchemy import *
from IPython.display import display

if os.path.isfile('BeStarsCatalog.db'):
    os.remove('BeStarsCatalog.db')
    

db = create_engine('sqlite:///BeStarsCatalog.db')
db.echo = False #

metadata = MetaData()

sabogalcatalog = Table('Sabogal',metadata,
                   Column('Type', String(3)),
                   Column('OGLE', String(16), primary_key=True),
                   Column('Vmag',Float(precision=2)),
                   Column('B-V', Float(precision=2)),
                   Column('V-I', Float(precision=2)),
                   Column('rms', Float(precision=2)),
                   Column('Per', Float(precision=2)),
                   Column('Per2', Float(precision=2)),)

zebruncatalog = Table('Zebrun',metadata,
                   Column('OGLE', String(16), primary_key=True),
                   Column('RAJ2000',Float(precision=6)),
                   Column('DEJ2000',Float(precision=6)),
                   Column('MC', String(1)),
                   Column('Field',Integer),
                   Column('Idat',Integer),
                   Column('OGLEII',String),
                   Column('Mag', Float(precision=4)),)

metadata.bind = db
metadata.create_all(db)

i = sabogalcatalog.insert()
j = zebruncatalog.insert()

## Be stars candidate in LMC ##

Catalogue made by Sabogal [(see more Sabogal et al. MNRAS (2005)](http://adsabs.harvard.edu/abs/2005MNRAS.361.1055S)). Acces to catalogue provide Vizier ID *J/MNRAS/361/1055/table1*. Before creation of SQL database we define conversion function for conversion between *Table* and *Dictionary* object

In [2]:
def tableRow_to_Dict(catalog):
    resultDict = {}
    resultDict['OGLE'] = catalog['OGLE'].data[0]
    resultDict['RAJ2000'] = catalog['RAJ2000'].data[0]
    resultDict['DEJ2000'] = catalog['DEJ2000'].data[0]
    resultDict['MC'] = catalog['MC'].data[0]
    resultDict['Field'] = int(catalog['Fld'].data[0])
    resultDict['Idat'] = int(catalog['Idat'].data[0])
    resultDict['OGLEII'] = str(catalog['ID2'].data[0])
    resultDict['Mag'] = catalog['mag2'].data[0]
    return resultDict

Finally, we have list of Be stars (using Zebrun catalogue *ID*). As the next step, we will query online *Vizier* database, which contains Zebrun catalog and download available information about stars with corresponding to the *ID*. All information are stored in the database

In [3]:
import warnings
from astroquery.vizier import Vizier
from sqlalchemy import exc

f = widgets.FloatProgress(min=0, max=2446,description='Database filling')
display(f)

Vizier.ROW_LIMIT = -1
catalog_sabogal = 'J/MNRAS/361/1055/table1'
catalog_zebrun = 'J/AcA/51/317/ogle2mc'

catalog = Vizier.get_catalogs(catalog_sabogal)

table = catalog[0]

warnings.filterwarnings(action='ignore')
warnings.simplefilter(action='ignore')
for counter,row in enumerate(table):
    try:
        i.execute({'Type': row['Type'],'OGLE': row['OGLE'],\
               'Vmag': row['Vmag'],'B-V': row['B-V'],'V-I': row['V-I'],\
               'rms': row['rms'],'Per': row['Per'],'Per2': row['Per2']})
        
        ogleid = '=='+str(row['OGLE'])
        result=Vizier(columns=['OGLE','RAJ2000','DEJ2000','MC','Fld','Idat','ID2','mag2']).query_constraints(catalog=catalog_zebrun,OGLE=ogleid)
        
        infozebrun = result[0]
#        print tableRow_to_Dict(infozebrun)
        j.execute(tableRow_to_Dict(infozebrun))
        f.value=counter
        
    except exc.IntegrityError:
        pass
        #print('Duplicite entry - Star  OGLE%s' % (str(row['OGLE'])))
print(ogleid)    
print('--------Catalog created=-------')


FloatProgress(value=0.0, description='Database filling', max=2446.0)

==05261038-6932021
--------Catalog created=-------


Output type of VIZIER query is object Astropy [*Table*](http://docs.astropy.org/en/stable/table/). SQLAlchemy store data to database using type [*dictionary*](https://www.tutorialspoint.com/python/python_dictionary.htm). Thus it is neccessary to convert object *Table* to type *dictionary*.